<a href="https://colab.research.google.com/github/Chienstartup/Tuna-Economics-and-its-Sustainability/blob/main/EDA_%26_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Library

In [2]:
pip install dataframe_image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 18.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import dataframe_image as  dfi
import plotly.express as px
from vega_datasets import data
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
from numpy.polynomial.polynomial import polyfit
from sklearn.linear_model import LinearRegression
import scipy.stats as stats

In [4]:
#altair limits 5000 rows for visualization, below code allows 10000 rows
from altair import pipe, limit_rows, to_values
t = lambda data: pipe(data, limit_rows(max_rows=10000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')

DataTransformerRegistry.enable('custom')

### Web Scarping for Coutry CSV

In [5]:
# since it require extra program, uncomment to run

# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup as soup

In [6]:
# #scraping url
# url = 'https://www.iban.com/country-codes'
# # launching selenium
# options = Options()
# options.add_argument("start-maximized")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [7]:
# get url
# driver.get(url)
# # parse page
# page_soup = soup(driver.page_source)
# # get columns head
# columns = []
# _col = page_soup.findAll('thead')[0].findAll('th')
# for i in _col:
#     columns.append(i.text)
# # get data table
# data = []
# _data = page_soup.findAll('tbody')[0].findAll('tr')
# for i in _data:
#     d = i.findAll('td')
#     _da = []
#     for j in d:
#         _da.append(j.text)
#     data.append(_da)
# # create dataframe
# df = pd.DataFrame(np.array(data),columns=columns)
# # write file to csv for ease of use in the future
# df.to_csv('assets/country_code.csv')

### Load Data

In [8]:
#load the fishery data set,
#main column that we are looking at:
#   ScieName: science name
#   Species: fish species abbreviation
#   SpeciesGrp: species group
#   YearC: year caught
#   Qty_t: quantity caught in tonnes
#   Flagname: country that caught

raw = pd.read_excel('https://raw.githubusercontent.com/Chienstartup/Tuna-Economics-and-its-Sustainability/main/data%20source/raw.xlsx', header = 3)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [9]:
# lets take a look at sample data from the data set
raw.head(2)

,RecID,Species,ScieName,SpeciesGrp,YearC,Decade,Lustrum,PartyStatus,PartyName,FlagName,...,ratio(%),Unnamed: 28,CnvFactorID,CfEquation,CfEquationType,Param_A,Param_B,SpeciesID,AuthorRef,Remarks
0,1,BFT,Thunnus thynnus,1-Tuna (major sp.),1950,1950,1950,CP,ALGÉRIE,Algerie,...,8.395167,NaN,<100,Various (B) (a=0),linear (y=bx),0.00,1=< b <= 2.4,0.0,none,none
1,2,BON,Sarda sarda,2-Tuna (small),1950,1950,1950,CP,ALGÉRIE,Algerie,...,83.139654,NaN,101,BET *1 WW=1.133*PW+2.980,linear (y=a+bx),2.98,1.133,5.0,pending,used by JPN


In [10]:
# load the second dataset, the size dataset
# this dataset contains catch data raised to total landing / location
# main column that we are looking at:
#   SpeciesCode: fish species abbreviation
#   YearC: year caught
#   yLat5ctoid: latitude that caught
#   xLon5ctoid: lontitude that caught
#   Catch_t: catch quantity in tonnes
size = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/Tuna-Economics-and-its-Sustainability/main/data%20source/sizemap.csv')

<ipython-input-10-d3b87e4e9902>:9: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  size = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/Tuna-Economics-and-its-Sustainability/main/data%20source/sizemap.csv')


In [11]:
# lets take a look at sample data from the data set
size.head(2)

,SpeciesCode,YearC,Decade,FlagName,FleetCode,Stock,GearGrp,SchoolType,Trimester,SampAreaCode,QuadID,Lat5,Lon5,yLat5ctoid,xLon5ctoid,CWPCode,pID,Catch_t
0,ALB,1950.0,1950.0,EU-España,EU.ESP-ES-CANT_ALB,ATN,TR,NaN,2.0,AL31,4.0,40.0,5.0,42.5,-7.5,644005,005W40N,12924.15311
1,ALB,1950.0,1950.0,EU-España,EU.ESP-ES-CANT_ALB,ATN,TR,NaN,3.0,AL31,4.0,40.0,5.0,42.5,-7.5,644005,005W40N,11627.35790


In [12]:
# load the third data set, the trade data set
# main column that we are looking at:
#   Country: country
#   Year: year
#   Commodity: type of goods
#   Flow: import, export or re-import
#   Value: Amount (USD)
trade = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/Tuna-Economics-and-its-Sustainability/main/data%20source/trade.csv')

<ipython-input-12-afff29d187a1>:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  trade = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/Tuna-Economics-and-its-Sustainability/main/data%20source/trade.csv')


In [13]:
# lets take a look at sample data from the data set
trade.head(2)

,COUNTRY,Country,COMMODITY,Commodity,FLOW,Flow,YEAR,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,TOTAL,Total trade of fisheries commodities,EXP,Exports,2010,2010,USD,US Dollar,0,Units,NaN,NaN,9.522360e+08,NaN,NaN
1,AUS,Australia,TOTAL,Total trade of fisheries commodities,EXP,Exports,2011,2011,USD,US Dollar,0,Units,NaN,NaN,1.008875e+09,NaN,NaN


In [14]:
# load the forth data set, the EU price data set of Tuna
# main column that we are looking at:
#   COUNTRY: country
#   YEAR: year
#   MAIN COMMERCIAL SPECIES: type of fish
#   VALUE (EUR).1: Export price in (EU)
#   VOLUME (KG).1: Export Quantity in KG
#   VALUE (EUR).2: Import price in (EU)
#   VOLUME (KG).2: Import Quantity in KG
eup = pd.read_excel('https://raw.githubusercontent.com/Chienstartup/Tuna-Economics-and-its-Sustainability/main/data%20source/EU_price.xlsx', header = 1)

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [15]:
# lets take a look at sample data from the data set
eup.head(2)

,COUNTRY,YEAR,MAIN COMMERCIAL SPECIES,VALUE (EUR),VOLUME (KG),VALUE (EUR).1,VOLUME (KG).1,VALUE (EUR).2,VOLUME (KG).2
0,Austria,2020.0,"Tuna, albacore",NaN,NaN,0.0,0.0,146000.0,8000.0
1,NaN,NaN,"Tuna, bigeye",NaN,NaN,0.0,0.0,43000.0,5000.0


In [16]:
# rename the columns for ease of use and understanding
eup = eup[['COUNTRY', 'YEAR', 'MAIN COMMERCIAL SPECIES', 'VALUE (EUR).1', 'VOLUME (KG).1', 'VALUE (EUR).2', 'VOLUME (KG).2']]
eup.columns = ['Country', 'YearC', 'Species', 'Exp_EUR','Exp_Kg', 'Imp_EUR','Imp_Kg']

In [17]:
#fill nan between value
eup= eup.ffill()

In [18]:
# lets take a look at clean epp
eup.head()

,Country,YearC,Species,Exp_EUR,Exp_Kg,Imp_EUR,Imp_Kg
0,Austria,2020.0,"Tuna, albacore",0.0,0.0,146000.0,8000.0
1,Austria,2020.0,"Tuna, bigeye",0.0,0.0,43000.0,5000.0
2,Austria,2020.0,"Tuna, bluefin",32000.0,0.0,239000.0,13000.0
3,Austria,2020.0,"Tuna, miscellaneous",3009000.0,445000.0,16399000.0,2653000.0
4,Austria,2020.0,"Tuna, skipjack",2701000.0,623000.0,39949000.0,7196000.0


In [19]:
# lets load the country code data
code = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/Tuna-Economics-and-its-Sustainability/main/data%20source/country_code.csv')

In [20]:
# we are only interest in Country and Alpha-3 code
code = code[['Country','Alpha-3 code']]
code.columns = ['Country','Country Code']

In [21]:
# lets take a look at the cleaned code df
code.head(2)

,Country,Country Code
0,Afghanistan,AFG
1,Albania,ALB


### <h1>EDA - Figure 1</h1>

In [22]:
#select relevant columns
raws = raw[['Species','SpeciesGrp', 'YearC', 'FlagName','Qty_t']]
#select major tuna species
major = raws[raws.SpeciesGrp.str.contains('1-Tuna ')]
maj = major[['Species','YearC', 'FlagName','Qty_t']].copy()
#aggregate caught volume
syq = maj.groupby(['Species','YearC'])['Qty_t'].sum().reset_index().round(1)
#select year > 2000
y2000 = syq[syq['YearC'] > 2000]

In [23]:
#plot line chart to show trend
line = alt.Chart(y2000).mark_line(size =5).encode(
    x = alt.X('YearC:N', title = 'Year'),
    y = alt.Y('Qty_t:Q', title = 'Tuna Caught Volume (Tons)'),
    color = alt.Color('Species:N')
).properties(
    width = 500,
    height = 350,
    title = 'Figure 1. Tuna Caught Volume for Each Species by Year in Tons'
).configure(
    background="#fafaf5"
).configure_title(
    fontSize = 20,
    offset = 10,
    color = '#0f1e5e',
).configure_axis(
    labelFontSize=13,
    titleFontSize=16,
).configure_legend(
    labelFontSize = 13,
    titleFontSize=14,
    symbolSize = 300,
    symbolStrokeWidth = 10
)


line

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### <h1>EDA - Figure 2</h1>

In [24]:
#ICCAT datasets use flagnames as country names, but some of the flagnames are different from the country name
#require replace the name for standardization
#hard coding
replace_name = [['Algerie','Algeria'],
 ['British Virgin Islands','Virgin Islands (British)'],
 ['Cape Verde','Cabo Verde'],
 ['Cayman Islands','Cayman Islands (the)'],
 ['China PR','China'],
 ['Chinese Taipei','Taiwan (Province of China)'],
 ['Congo','Congo (the Democratic Republic of the)'],
 ['Dominican Republic','Dominican Republic (the)'],
 ['FR-St Pierre et Miquelon','Saint Pierre and Miquelon'],
 ['Faroe Islands','Faroe Islands (the)'],
 ['Gambia','Gambia (the)'],
 ['Great Britain','United Kingdom of Great Britain and Northern Ireland (the)'],
 ['Guinea Ecuatorial','Guinea'],
 ['Guinée Rep','Guinea-Bissau'],
 ['ICCAT (RMA)','others'],
 ['Korea Rep','Korea (the Republic of)'],
 ['Maroc','Morocco'],
 ['Mixed flags (EU tropical)','others'],
 ['Mixed flags (FR+ES)','others'],
 ['NEI (BIL)','others'],
 ['NEI (ETRO)','others'],
 ['NEI (Flag related)','others'],
 ['NEI (MED)','others'],
 ['NEI (combined)','others'],
 ['NEI (inflated)','others'],
 ['Netherlands','Netherlands (the)'],
 ['Philippines','Philippines (the)'],
 ['Rumania','Romania'],
 ['Russian Federation','Russian Federation (the)'],
 ['S Tomé e Príncipe','Sao Tome and Principe'],
 ['Serbia & Montenegro','Serbia'],
 ['St Vincent and Grenadines','Saint Vincent and the Grenadines'],
 ['Sta Helena','Saint Helena, Ascension and Tristan da Cunha'],
 ['Sta Lucia', 'Saint Lucia'],
 ['Syria','Syrian Arab Republic'],
 ['Tunisie','Tunisia'],
 ['Turks and Caicos','Turks and Caicos Islands (the)'],
 ['USA','United States of America (the)'],
 ['USSR','others'],
 ['Venezuela','Venezuela (Bolivarian Republic of)'],
 ['Yugoslavia Fed','others'],
 ['spaña','Spain'],
 ['stonia','Estonia'],
 ['EU-France','France'],
 ['EU-Greece','Greece'],
 ['EU-Portugal','Portugal'],
 ['EU-Malta','Malta'],
 ['EU-Germany','Germany'],
 ['EU-España','Spain'],
 ['EU-Sweden','Sweden'],
 ['UK-Sta Helena','Saint Helena, Ascension and Tristan da Cunha'],
 ['EU-Poland','Poland'],
 ['EU-Cyprus','Cyprus'],
 ['EU-Ireland','Ireland'],
 ['UK-Bermuda','Bermuda'],
 ['EU-Bulgaria','Bulgaria'],
 ['EU-Rumania','Romania'],
 ['EU-Denmark','Denmark'],
 ['EU-Croatia','Croatia'],
 ['EU-Estonia','Estonia'],
 ['EU-Latvia','Latvia'],
 ['EU-Lithuania','Lithuania'],
 ['UK-Turks and Caicos', 'Turks and Caicos Islands (the)'],
 ['UK-British Virgin Islands','Virgin Islands (British)'],
 ['EU-Netherlands','Netherlands (the)'],
 ['EU-Italy','Italy']]
#change the dataframe with list and for-loop
for country in replace_name:
    maj.replace(country[0],country[1],inplace=True)

In [25]:
#focus tuna list
focus_tuna = ['SKJ','BET','YFT','ALB','BFT']
#setting join string to filter tuna species
tuna_str = '|'.join(focus_tuna)
foc_maj = maj[maj['Species'].str.contains(tuna_str)].reset_index(drop=True)
fm_vis = foc_maj.groupby(['FlagName'])['Qty_t'].sum().reset_index().round(0)
#sorting dataframe and select top 21 (including one other group, which will be remove by dropna)
top21 = fm_vis.sort_values(by = 'Qty_t',ascending=False).head(21)
top21.columns = ['Country', 'Caught_Volume']
topcode = top21.merge(code, how = 'left', on = 'Country').dropna()
# let look at the top 5 of the top 21 dataframe
topcode.head(5)

,Country,Caught_Volume,Country Code
0,Spain,6137891.0,ESP
1,France,3748634.0,FRA
2,Japan,2661238.0,JPN
3,Ghana,2304264.0,GHA
4,Taiwan (Province of China),1800058.0,TWN


In [26]:
#draw bar chart
alt.Chart(topcode).mark_bar(color = '#1180f7').encode(
    x = alt.X('Country Code:N',
              sort = alt.EncodingSortField('Caught_Volume', order = 'descending'),#sort the x axis by caught volume
              title = 'Country'
             ),
    y = alt.Y('Caught_Volume:Q',
               title = 'Historical Total Tuna Caught Volume (Tons)'),
).properties(
    width = 500,
    height = 350,
    title = 'Figure 2. Historical Total Tuna Caught Volume by Country: Top 20'
).configure(
    background="#fafaf5"
).configure_title(
    fontSize = 20,
    offset = 10,
    color = '#0f1e5e',
).configure_axis(
    labelFontSize=13,
    titleFontSize=16,
).configure_legend(
    labelFontSize = 13,
    titleFontSize=14,
    symbolSize = 300,
    symbolStrokeWidth = 10
)

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### <h1>Analysis 1 </h1>

In [27]:
#replace tuna name with tuna code
fish_name = [['Tuna, albacore', 'ALB'],
             ['Tuna, bigeye', 'BET'],
             ['Tuna, bluefin', 'BFT'],
             ['Tuna, miscellaneous', 'other'],
             ['Tuna, skipjack', 'SKJ'],
             ['Tuna, yellowfin','YFT']]

for fish in fish_name:
    eup.replace(fish[0],fish[1],inplace=True)

In [28]:
#focus tuna list
focus_tuna = ['SKJ','BET','YFT','ALB','BFT']
#setting join string to filter tuna species
tuna_str = '|'.join(focus_tuna)
eup = eup[eup['Species'].str.contains(tuna_str)].reset_index(drop=True)

In [29]:
#calculate the average EUR price of tuna species per Kg
agg_eup = eup.groupby(['Species', 'YearC']).sum().reset_index()
agg_eup['total_EUR'] = agg_eup['Exp_EUR'] + agg_eup['Imp_EUR']
agg_eup['total_Kg'] = agg_eup['Exp_Kg'] + agg_eup['Imp_Kg']
agg_eup['Exp_EUR_per_Kg'] = agg_eup['Exp_EUR']/agg_eup['Exp_Kg']
agg_eup['Imp_EUR_per_Kg'] = agg_eup['Imp_EUR']/agg_eup['Imp_Kg']
agg_eup['EUR_per_Kg'] = agg_eup['total_EUR']/agg_eup['total_Kg']

In [30]:
#prepare agg_maj from raw.xlsx
raws = raw[['Species','SpeciesGrp', 'YearC', 'FlagName','Qty_t']]
major = raws[raws.SpeciesGrp.str.contains('1-Tuna ')]
maj = major[['Species','YearC', 'FlagName','Qty_t']].copy()
agg_maj = maj.groupby(['Species','YearC','FlagName'])['Qty_t'].sum().reset_index().round(0)
agg_maj.head(5)

,Species,YearC,FlagName,Qty_t
0,ALB,1950,EU-España,24943.0
1,ALB,1950,EU-France,14680.0
2,ALB,1950,EU-Malta,0.0
3,ALB,1950,South Africa,0.0
4,ALB,1951,EU-España,19446.0


In [31]:
#raw (agg_maj)
agg_maj_qty = agg_maj.groupby(['Species', 'YearC'])['Qty_t'].sum().reset_index()

In [32]:
#combine eu price and agg_maj
combine_eup_gty = agg_eup.merge(agg_maj_qty, how = 'left', on = ['Species', 'YearC']).dropna()
combine_eup_gty = combine_eup_gty[combine_eup_gty['YearC'] > 2009]
corr_eup = combine_eup_gty[['EUR_per_Kg', 'Exp_EUR_per_Kg','Imp_EUR_per_Kg','Qty_t']].corr().round(2)
#export dataframe as png
corr_styled = corr_eup.style.background_gradient()
#uncomment below to export png
 # dfi.export(corr_styled , 'corr_styled.png')

In [33]:
#corr for bft
bft = combine_eup_gty[combine_eup_gty['Species'] =='BFT']
bft_eup = bft[['EUR_per_Kg', 'Exp_EUR_per_Kg','Imp_EUR_per_Kg','Qty_t']].corr().round(2)
bft_styled = bft_eup.style.background_gradient()
#uncomment below to export png
 # dfi.export(bft_styled , 'bft_styled.png')

#### Figure 4

In [34]:
subjects = ['EUR_per_Kg', 'Exp_EUR_per_Kg','Imp_EUR_per_Kg']

#plot the scatter plot and the regression line with R-square
def module(subjects):
    chart = alt.vconcat()
    for sub in subjects:
        base = alt.Chart(combine_eup_gty).mark_circle().encode(
            x = alt.X('Qty_t:Q', title ='Tuna Caught Volume (Tons)'),
            y = alt.Y('{}:Q'.format(sub), title = '{}'.format(sub) )
        ).properties(
            width = 200,
            height = 150
        )

        line = base.transform_regression('Qty_t', sub ).mark_line()

        params = base.transform_regression('Qty_t', sub,
                                     params=True
        ).mark_text(align='left', lineBreak='\n'
        ).encode(
            x=alt.value(150),  # pixels from left
            y=alt.value(50),  # pixels from top
            text='params:N'
        ).transform_calculate(
            params='"R² = " + round(datum.rSquared * 100)/100'
        )

        chart |= (base + line + params)

    return chart.properties(
        title = 'Figure 4. Tuna Caught Volume vs EUR per Kg (Different Flow Type)'
    ).configure(
        background="#fafaf5"
    ).configure_title(
        fontSize = 20,
        offset = 10,
        color = '#0f1e5e',
        subtitleFontSize =17,
    )


In [35]:
module(subjects)

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.c

alt.HConcatChart(...)

#### Figure 3

In [36]:
#price for tuna species
base = alt.Chart(combine_eup_gty).mark_line(size = 3).encode(
    x = alt.X('YearC:N',title='Year'),
    y = 'EUR_per_Kg:Q',
    color = 'Species:N'
)


#text of value in 2020
text = (
    alt.Chart(combine_eup_gty)
    .mark_text(size = 12,
               align = 'left',
            baseline = 'middle'
              )
    .transform_filter((alt.datum.YearC == 2020))
    .encode(x=alt.X('YearC:N'),
            y=alt.Y('EUR_per_Kg:Q'),
            text=alt.Text("EUR_per_Kg:Q", format='.2f'))
)


(base + text).properties(
        width=500,
        height = 350,
        title = 'Figure 3. EUR price per Kg for five major tuna species'
    ).configure(
        background="#fafaf5"
    ).configure_title(
        fontSize = 20,
        offset = 10,
        color = '#0f1e5e',
        subtitleFontSize =17,
).configure_axis(
    labelFontSize=13,
    titleFontSize=16,
).configure_legend(
    labelFontSize = 13,
    titleFontSize=14,
    symbolSize = 300,
    symbolStrokeWidth = 10
)

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

### <h1>Analysis 2</h1>

In [53]:
#load csv
gdpo = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/Tuna-Economics-and-its-Sustainability/main/data%20source/GDP_oecd.csv')

In [54]:
#select relevant columns
oecd = gdpo[['LOCATION', 'Country','Year', 'Value']]

In [55]:
#leave only country data
m_oecd = oecd[(oecd.Country != 'Euro area (19 countries)') &
     (oecd.Country != 'OECD - Total') &
    (oecd.Country != 'Euro area (19 countries)') &
     (oecd.Country != 'European Union – 27 countries (from 01/02/2020)') ].round(0)

In [56]:
#rename column to match the code with raws.xlsx (foc_maj dataframe from EDA part)
m_oecd.columns = ['Country Code', 'Country', 'YearC', 'GDP per capita']
# let take a sample look at the data for Spain
m_oecd[m_oecd['Country'] == 'Spain']

,Country Code,Country,YearC,GDP per capita
121,ESP,Spain,2010,31693.0
122,ESP,Spain,2011,31872.0
123,ESP,Spain,2012,31725.0
124,ESP,Spain,2013,32453.0
125,ESP,Spain,2014,33544.0
126,ESP,Spain,2015,34929.0
127,ESP,Spain,2016,37314.0
128,ESP,Spain,2017,39580.0
129,ESP,Spain,2018,40756.0
130,ESP,Spain,2019,41721.0


In [57]:
#renam column for matching
foc_maj_oecd = foc_maj.copy()
foc_maj_oecd.columns = ['Species', 'YearC', 'Country', 'Qty_t']
#change country name in foc_maj_oecd
for country in replace_name:
    foc_maj_oecd.replace(country[0],country[1],inplace=True)
# merge with code
foc_maj_merge = foc_maj_oecd.merge(code, how = 'left', on = 'Country')

In [58]:
#combine two datasets ( gdp and tuna caught volume)
combine_catch_gdp = foc_maj_merge.merge(m_oecd, how = 'left', on = ['YearC', 'Country Code']).dropna()
stat_catch_gdp = combine_catch_gdp.groupby(['YearC', 'Country Code']).agg({'Qty_t':'sum', 'GDP per capita':'mean'}).reset_index()
corr_gdp = stat_catch_gdp[['GDP per capita', 'Qty_t']].corr().round(2)
#export dataframe as png
corr_gdp_styled = corr_gdp .style.background_gradient()
#uncomment below to export png
 # dfi.export(corr_gdp_styled  , 'corr_gdp_styled.png')
stat_catch_gdp['Country Code'].unique()

array(['ARG', 'BRA', 'CAN', 'CHN', 'CYP', 'ESP', 'FRA', 'GBR', 'GRC',
       'HRV', 'IRL', 'ITA', 'JPN', 'KOR', 'LVA', 'MEX', 'MLT', 'PRT',
       'TUR', 'USA', 'ZAF', 'ISL', 'NLD', 'RUS', 'NOR', 'LTU', 'DEU',
       'DNK'], dtype=object)

#### Figure 5

In [59]:
#scatter plot linear regression with r-square
chart = alt.Chart(stat_catch_gdp).mark_circle().encode(
    x = alt.X('Qty_t:Q', title ='Tuna Caught Volume (Tons)'),
    y = alt.Y('GDP per capita:Q', title = 'GDP per capita (US$)' )
)

line = chart.transform_regression('Qty_t', 'GDP per capita').mark_line()

params = chart.transform_regression('Qty_t', 'GDP per capita',
                                     params=True
).mark_text(align='left', lineBreak='\n'
).encode(
    x=alt.value(150),  # pixels from left
    y=alt.value(100),  # pixels from top
    text='params:N'
).transform_calculate(
    params='"R² = " + round(datum.rSquared * 100)/100'
            )


(chart + line + params).properties(
    title = 'Figure 5. Tuna Caught Volume vs GDP per capita: OECD Countries'
).configure(
    background="#fafaf5"
).configure_title(
    fontSize = 16,
    offset = 10,
    color = '#0f1e5e',
    subtitleFontSize =17,
)

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

#### Figure 6

In [60]:
#select Spain, France, and Japan
foc_country =stat_catch_gdp[(stat_catch_gdp['Country Code'].str.contains('JPN'))|
               (stat_catch_gdp['Country Code'].str.contains('FRA'))|
              (stat_catch_gdp['Country Code'].str.contains('ESP'))]
#run corr again
sel_gdp = foc_country[['GDP per capita', 'Qty_t']].corr().round(2)
sel_gdp_styled = sel_gdp .style.background_gradient()
#uncomment below to export png
 # dfi.export(sel_gdp_styled  , 'sel_gdp_styled.png')

In [61]:
#scatter plot linear regression with r-square
chart = alt.Chart(foc_country).mark_circle().encode(
    x = alt.X('Qty_t:Q', title ='Tuna Caught Volume (Tons)'),
    y = alt.Y('GDP per capita:Q', title = 'GDP per capita (US$)' )
)

line = chart.transform_regression('Qty_t', 'GDP per capita').mark_line()

params = chart.transform_regression('Qty_t', 'GDP per capita',
                                     params=True
).mark_text(align='left', lineBreak='\n'
).encode(
    x=alt.value(150),  # pixels from left
    y=alt.value(100),  # pixels from top
    text='params:N'
).transform_calculate(
    params='"R² = " + round(datum.rSquared * 100)/100'
            )


(chart + line + params).properties(
    title = 'Figure 6. Tuna Caught Volume vs GDP per capita: Spain, France, and Japan'
).configure(
    background="#fafaf5"
).configure_title(
    fontSize = 15,
    offset = 10,
    color = '#0f1e5e',
    subtitleFontSize =17,
)

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

### <h1>Analysis 3</h1>

In [62]:
#trade dataset
trade.head()
trade['Commodity'] = trade['Commodity'].str.lower()
#filter tuna dataset
tuna_trade = trade[trade['Commodity'].str.contains('tuna')].copy()

In [63]:
#standardize the species names of two datasets (trade and caught)
replace_tuna = [['bluefin tunas (thunnus thynnus, thunnus orientalis), live','BFT'],
 ['atlantic and pacific southern bluefin tunas (thunnus maccoyii), live','other'],
 ['albacore or longfinned tuna (thunnus alalunga), fresh or chilled','ALB'],
 ['yellowfin tunas (thunnus albacares), fresh or chilled','YFT'],
 ['bigeye tuna, fresh or chilled','BET'],
 ['atlantic and pacific bluefin tuna (thunnus thynnus, thunnus orientalis), fresh or chilled','BFT'],
 ['southern bluefin tuna (thunnus maccoyii), fresh or chilled','other'],
 ['other tunas, fresh or chilled','other'],
 ['albacore or longfinned tunas (thunnus alalunga), frozen','ALB'],
 ['yellowfin tunas (thunnus albacares), frozen','YFT'],
 ['bigeye tuna, frozen','BET'],
 ['atlantic and pacific bluefin tuna (thunnus thynnus, thunnus orientalis), frozen','BFT'],
 ['southern bluefin tuna (thunnus maccoyii), frozen','other'],
 ['other tunas, frozen','other'],
 ['tunas, skipjack or stripe-bellied bonito, fillets, frozen','SKJ'],
 ['tunas, skipjack and atlantic bonito, prepared or preserved','SKJ'],]
#change the dataframe with list and for-loop
tuna_trade['Commodity'] = tuna_trade['Commodity'].str.lower()
for tuna in replace_tuna:
    tuna_trade.replace(tuna[0],tuna[1],inplace=True)

In [64]:
#select useful columns
tuna_trade_red = tuna_trade[['COUNTRY', 'Country','Commodity', 'Flow', 'Year',  'Value']].reset_index(drop = True)

In [65]:
#rename column
tuna_trade_red.columns = ['Country Code', 'Country', 'Species', 'Flow', 'YearC', 'Value']
tuna_trade_red.head()

,Country Code,Country,Species,Flow,YearC,Value
0,AUS,Australia,BFT,Exports,2012,359000.0
1,AUS,Australia,BFT,Exports,2013,277000.0
2,AUS,Australia,BFT,Exports,2014,478000.0
3,AUS,Australia,BFT,Exports,2015,761000.0
4,AUS,Australia,BFT,Exports,2016,1564000.0


In [66]:
#calculate the net import flow
net_flow = tuna_trade_red.groupby(['Country Code', 'Species', 'YearC', 'Flow'])['Value'].aggregate('first').unstack().reset_index()
net_flow = net_flow.fillna(0)
#net import flow = import - export - re-exports
net_flow['net_imp_flow'] = net_flow['Imports'] - net_flow['Exports'] - net_flow['Re-Exports']
net_flow['total_market_value'] = net_flow['Imports']+ net_flow['Exports'] + net_flow['Re-Exports']
#focus tuna list
focus_tuna = ['SKJ','BET','YFT','ALB','BFT']
#setting join string to filter tuna species
tuna_str = '|'.join(focus_tuna)
net_flow = net_flow[net_flow['Species'].str.contains(tuna_str)].reset_index(drop=True)

In [67]:
#merge tuna caught dataframe foc_maj_merge from analysis 2
combine_flow_catch = net_flow.merge(foc_maj_merge, how='left', on = ['Species', 'YearC', 'Country Code']).dropna()

In [68]:
combine_flow_catch.head()

,Country Code,Species,YearC,Exports,Imports,Re-Exports,net_imp_flow,total_market_value,Country,Qty_t
0,ARG,ALB,2010,0.0,4000.0,0.0,4000.0,4000.0,Argentina,129.558
1,ARG,ALB,2011,0.0,1000.0,0.0,1000.0,1000.0,Argentina,43.186
8,ARG,SKJ,2010,0.0,40151000.0,0.0,40151000.0,40151000.0,Argentina,11.937
9,ARG,SKJ,2011,0.0,59706000.0,0.0,59706000.0,59706000.0,Argentina,0.064
19,ARG,YFT,2010,0.0,30000.0,0.0,30000.0,30000.0,Argentina,0.018


In [69]:
#agg with sum
agg_flow_catch = combine_flow_catch.groupby(['Country Code', 'YearC']).agg({'net_imp_flow':'sum', 'total_market_value':'sum', 'Qty_t':'sum'}).reset_index().round(0)
vis_flow_catch = agg_flow_catch.sort_values(by = 'total_market_value', ascending = False)
vis_flow_catch['net_imp_flow'] =  vis_flow_catch['net_imp_flow'] /1000000
vis_flow_catch['total_market_value'] =  vis_flow_catch['total_market_value'] /1000000
vis_flow_catch_y  = vis_flow_catch[vis_flow_catch['YearC'] > 2019 ].head(5)
vis_flow_catch_y = vis_flow_catch_y.round(0)


In [70]:
vis_flow_catch.head()

,Country Code,YearC,net_imp_flow,total_market_value,Qty_t
112,JPN,2014,14472.051000,15614.237000,20264.0
117,JPN,2019,13439.682011,15596.667866,18207.0
113,JPN,2015,12575.469000,13552.079000,19624.0
116,JPN,2018,12692.166000,13392.640000,18840.0
111,JPN,2013,11931.276000,12645.570000,24277.0


#### Figure 7

In [71]:
#show the market size and flow
base = alt.Chart(vis_flow_catch_y).mark_circle().encode(
    x = alt.X('net_imp_flow:Q', title = 'Net Import Flow (US$M)'),
    y = alt.Y('total_market_value:Q', title = 'Tuna Market Value (US$M)'),
    color = alt.Color('Country Code:N', legend = None),
    size = alt.Size('Qty_t:Q', scale = alt.Scale(range = (100,500)), legend = None)
)

text = (
    alt.Chart(vis_flow_catch_y)
    .mark_text(size = 14,
               align = 'center',
            baseline = 'bottom',
               dy = -18,
               dx = 27
              )
    .transform_filter((alt.datum.YearC == 2020))
    .encode(x=alt.X('net_imp_flow:Q'),
            y=alt.Y('total_market_value:Q'),
            text=alt.Text("Country Code:N"))
)

text2 = (
    alt.Chart(vis_flow_catch_y)
    .mark_text(size = 14,
               align = 'center',
            baseline = 'top',
               dy = -18,
               dx = 27
              )
    .transform_filter((alt.datum.YearC == 2020))
    .encode(x=alt.X('net_imp_flow:Q'),
            y=alt.Y('total_market_value:Q'),
            text=alt.Text("Qty_t:Q", format = '.0f')
           ).transform_calculate(label='datum.Qty_t + " Tones"')
)


(base + text + text2).properties(
    title = 'Figure 7. Top 5 Tuna Markets: Net Import Flow and Tuna Caught Volume'
).configure(
    background="#fafaf5"
).configure_title(
    fontSize = 15,
    offset = 10,
    color = '#0f1e5e',
    subtitleFontSize =17,
)

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

#### Figure 8

In [72]:
#focus country list
focus_country = ['ESP','JPN','USA','KOR','FRA']
#setting join string to filter countries
country_str = '|'.join(focus_country)
vis_flow_catch_country = vis_flow_catch[vis_flow_catch['Country Code'].str.contains(country_str)].reset_index(drop=True)

In [73]:
#combine trade and gdp
combine_imp_gdp = vis_flow_catch_country.merge(stat_catch_gdp, how = 'left', on=['YearC','Country Code']).round(0)
combine_imp_gdp.head()

,Country Code,YearC,net_imp_flow,total_market_value,Qty_t_x,Qty_t_y,GDP per capita
0,JPN,2014,14472.0,15614.0,20264.0,20264.0,39560.0
1,JPN,2019,13440.0,15597.0,18207.0,18207.0,42439.0
2,JPN,2015,12575.0,13552.0,19624.0,19624.0,40909.0
3,JPN,2018,12692.0,13393.0,18840.0,18840.0,42239.0
4,JPN,2013,11931.0,12646.0,24277.0,24277.0,39437.0


In [74]:
countries= ['ESP','JPN','USA','KOR','FRA']

def focused_country(countries):
    total = alt.vconcat()

    for country in countries:
        df = combine_imp_gdp[combine_imp_gdp['Country Code'] == country]
        chart = alt.Chart(df).mark_circle().encode(
            x = alt.X('net_imp_flow:Q', title ='Tuna Import Value (US$M)'),
            y = alt.Y('GDP per capita:Q', title = 'GDP per capita (US$)' )
        ).properties(width = 150, height = 120)

        line = chart.transform_regression('net_imp_flow', 'GDP per capita').mark_line()

        text = alt.Chart(df).mark_text(
                size = 14,
                   align = 'center',
                baseline = 'bottom',
              ).transform_filter((alt.datum.YearC == 2015)).encode(x=alt.X('net_imp_flow:Q'),
            y=alt.Y('GDP per capita:Q'),
            text=alt.Text("Country Code:N"))


        params = chart.transform_regression('net_imp_flow', 'GDP per capita',
                                     params=True
        ).mark_text(align='left', lineBreak='\n'
        ).encode(
            x=alt.value(100),  # pixels from left
            y=alt.value(100),  # pixels from top
            text='params:N'
        ).transform_calculate(
            params='"R² = " + round(datum.rSquared * 100)/100'
        )

        total |= (chart + line + params+text)

    return total.properties(
        title = 'Figure 8. GDP per capita vs Tuna Import Value'
    ).configure(
        background="#fafaf5"
    ).configure_title(
        fontSize = 15,
        offset = 10,
        color = '#0f1e5e',
        subtitleFontSize =17,
    )
focused_country(countries)

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply

alt.HConcatChart(...)

In [75]:
#create five correlation
for country in countries:
    dff = combine_imp_gdp[combine_imp_gdp['Country Code'] == country]
    cor_country = dff[['net_imp_flow', 'GDP per capita']].corr().round(2)
    cor_single = cor_country.style.background_gradient()
    #uncomment below to export png
     # dfi.export(cor_single  , '{}_styled.png'.format(country))


### <h1>Analysis 4: overfishing</h1>

#### Figure 9

In [76]:
#sizemap dataset: select columns
reduce_col = size[['SpeciesCode', 'YearC', 'FlagName','yLat5ctoid', 'xLon5ctoid','Catch_t']].copy()

#focus tuna list
focus_tuna = ['SKJ','BET','YFT','ALB','BFT']
#setting join string to filter tuna species
tuna_str = '|'.join(focus_tuna)
reduce_col = reduce_col[reduce_col['SpeciesCode'].str.contains(tuna_str)].reset_index(drop=True)


#group by location and year
location_catch = reduce_col.groupby(['yLat5ctoid', 'xLon5ctoid', 'YearC'])['Catch_t'].sum().reset_index()
location_catch.columns = ['lat','lon', 'year', 'catch']

In [77]:
#set year 2000 as the benmark
location_catch = location_catch[location_catch['year'] > 1999]
agg_loc_catch = location_catch.groupby(['lat','lon','year'])['catch'].aggregate('first').unstack().reset_index().dropna()
vis_loc_catch = agg_loc_catch[[ 'lat',  'lon', 2000.0, 2020.0]].copy()
vis_loc_catch['difference'] = vis_loc_catch[2020.0] - vis_loc_catch[2000.0]
vis_loc_catch.columns = ['lat', 'lon', '2000', '2020', 'difference']

In [78]:
#draw world map
world = alt.topo_feature(data.world_110m.url, feature='countries')


background = alt.Chart(world).mark_geoshape(
            fill='lightgray',
            stroke='white'
            ).properties(
            width=500,
            height=350
            )

# mark point on world map based on location
points = alt.Chart(vis_loc_catch).mark_circle().encode(
            latitude='lat',
            longitude='lon',
            color = alt.Color('difference:Q', scale = alt.Scale(domain = (vis_loc_catch.difference.min(),vis_loc_catch.difference.max()) ,scheme='lightmulti'))
        )

(background + points).properties(
    title = {
      "text": ["Figure 9. Tuna Caught Volume at Same Location"],
      "subtitle": [" Difference between Year 2020 and 2000",'The closer the color is to red, the more serious the overfishing might be'],
    }
).configure(
    background="#fafaf5"
).configure_title(
    fontSize = 20,
    offset = 10,
    color = '#0f1e5e',
    subtitleFontSize =17,
)

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(


alt.LayerChart(...)

#### Figure 10

In [82]:
#load avg_catch_size file
catch_size = pd.read_excel('https://raw.githubusercontent.com/Chienstartup/Tuna-Economics-and-its-Sustainability/main/data%20source/avg_catch_size.xlsx', header = 3)

In [83]:
#select columns
catch_size_red_col = catch_size[['FlagName','YearC', 'SpeciesCode', 'SpeciesGrp', 'MinClass', 'MaxClass', 'MeanSize', 'NrRecs', 'NrFish']].copy()
#focus tuna list
focus_tuna = ['SKJ','BET','YFT','ALB','BFT']
#setting join string to filter tuna species
tuna_str = '|'.join(focus_tuna)
catch_size_red_col = catch_size_red_col[catch_size_red_col['SpeciesCode'].str.contains(tuna_str)].reset_index(drop=True)

catch_size_red_col.head(5)

,FlagName,YearC,SpeciesCode,SpeciesGrp,MinClass,MaxClass,MeanSize,NrRecs,NrFish
0,EU-Italy,1932,BFT,1-Tuna (major sp.),2,61,12.320807,33,3619.0
1,EU-Italy,1933,BFT,1-Tuna (major sp.),2,10,5.740601,9,3963.0
2,EU-Italy,1934,BFT,1-Tuna (major sp.),2,61,4.497986,10,3972.0
3,EU-Italy,1935,BFT,1-Tuna (major sp.),2,61,12.963116,60,2657.0
4,EU-España,1950,BFT,1-Tuna (major sp.),30,260,131.075630,43,595.0


In [84]:
#filter major tuna species
catch_size_maj = catch_size_red_col [catch_size_red_col ['SpeciesGrp'].str.contains('1-Tuna')]
catch_size_maj  = catch_size_maj[['FlagName', 'YearC', 'SpeciesCode',  'MinClass', 'MaxClass', 'MeanSize', 'NrRecs', 'NrFish']]
catch_size_maj.columns = ['FlagName', 'YearC', 'Species', 'MinClass',  'MaxClass', 'MeanSize', 'NrRecs', 'NrFish']
#prepare number for calculating weighted average size
catch_size_maj['WA_MeanSize'] = catch_size_maj['MeanSize'] * catch_size_maj['NrRecs']
catch_size_maj.head(5)

,FlagName,YearC,Species,MinClass,MaxClass,MeanSize,NrRecs,NrFish,WA_MeanSize
0,EU-Italy,1932,BFT,2,61,12.320807,33,3619.0,406.586626
1,EU-Italy,1933,BFT,2,10,5.740601,9,3963.0,51.665405
2,EU-Italy,1934,BFT,2,61,4.497986,10,3972.0,44.979859
3,EU-Italy,1935,BFT,2,61,12.963116,60,2657.0,777.786978
4,EU-España,1950,BFT,30,260,131.075630,43,595.0,5636.252101


In [85]:
agg_catch_size = catch_size_maj.groupby(['FlagName', 'YearC', 'Species']).agg({'MinClass':'min',  'MaxClass':'max', 'WA_MeanSize':'sum', 'NrRecs':'sum', 'NrFish':'sum'}).reset_index()
agg_catch_size.head(5)

,FlagName,YearC,Species,MinClass,MaxClass,WA_MeanSize,NrRecs,NrFish
0,Albania,2011,BFT,100,100,101.000000,1,4.0
1,Albania,2013,BFT,100,100,101.000000,1,4.0
2,Albania,2014,BFT,107,250,13438.909091,72,132.0
3,Albania,2015,BFT,128,228,7300.524590,37,61.0
4,Albania,2016,BFT,92,215,9449.410788,73,241.0


In [86]:
#prepare agg_maj from raw.xlsx
raws = raw[['Species','SpeciesGrp', 'YearC', 'FlagName','Qty_t']]
major = raws[raws.SpeciesGrp.str.contains('1-Tuna ')]
maj = major[['Species','YearC', 'FlagName','Qty_t']].copy()
agg_maj = maj.groupby(['Species','YearC','FlagName'])['Qty_t'].sum().reset_index().round(0)
agg_maj.head(5)

,Species,YearC,FlagName,Qty_t
0,ALB,1950,EU-España,24943.0
1,ALB,1950,EU-France,14680.0
2,ALB,1950,EU-Malta,0.0
3,ALB,1950,South Africa,0.0
4,ALB,1951,EU-España,19446.0


In [87]:
#join two datasets by left join on three keys
volume_size = agg_catch_size.merge(agg_maj, how = 'left', on =['FlagName','YearC','Species']).dropna()
volume_size.head(5)

,FlagName,YearC,Species,MinClass,MaxClass,WA_MeanSize,NrRecs,NrFish,Qty_t
0,Albania,2011,BFT,100,100,101.000000,1,4.0,0.0
1,Albania,2013,BFT,100,100,101.000000,1,4.0,9.0
2,Albania,2014,BFT,107,250,13438.909091,72,132.0,34.0
3,Albania,2015,BFT,128,228,7300.524590,37,61.0,40.0
4,Albania,2016,BFT,92,215,9449.410788,73,241.0,47.0


In [88]:
#calculate the weighted average size
agg_vis = volume_size.groupby(['YearC','Species']).agg({'MinClass':'min','MaxClass':'max', 'WA_MeanSize':'sum', 'NrRecs':'sum','Qty_t':'sum'}).reset_index()
agg_vis['Weighted_Average_Size'] = agg_vis['WA_MeanSize']/agg_vis['NrRecs']
df_vis = agg_vis[['MinClass', 'MaxClass', 'Weighted_Average_Size', 'Qty_t']].corr().round(2)
#export dataframe as png
df_styled = df_vis.style.background_gradient()
#uncomment below to export png
# dfi.export(df_styled, 'df_styled.png')

In [89]:
#plot the scatter plot and the regression line with R-square
chart = alt.Chart(agg_vis).mark_circle().encode(
    x = alt.X('Qty_t:Q', title ='Tuna Caught Volume (Tons)'),
    y = alt.Y('Weighted_Average_Size:Q', title = 'Weighted Average Tuna Size' )
)

line = chart.transform_regression('Qty_t', 'Weighted_Average_Size').mark_line()

params = chart.transform_regression('Qty_t', 'Weighted_Average_Size',
                                     params=True
).mark_text(align='left', lineBreak='\n'
).encode(
    x=alt.value(150),  # pixels from left
    y=alt.value(100),  # pixels from top
    text='params:N'
).transform_calculate(
    params='"R² = " + round(datum.rSquared * 100)/100'
            )


(chart + line + params).properties(
    title = 'Figure 10. Tuna Caught Volume vs Weighted Average Tuna Size'
).configure(
    background="#fafaf5"
).configure_title(
    fontSize = 17,
    offset = 10,
    color = '#0f1e5e',
    subtitleFontSize =17,
)

/usr/local/lib/python3.10/dist-packages/altair/utils/data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)